# Edgar Q&A Demo

In [ ]:
import os
import sys
from pprint import pprint

sys.path.append("../../")

from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA, LLMChain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory, ConversationSummaryMemory, ChatMessageHistory
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

from utils.sambanova_endpoint import SambaNovaEndpoint

from dotenv import load_dotenv
load_dotenv('../../export.env')

True

In [2]:
PERSIST_DIRECTORY = '../data/vectordbs/tsla'

### Load embedding model and SEC vector db

In [3]:
embedding_model = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the query for retrieval: "
)
vectordb = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embedding_model)

/opt/homebrew/Caskroom/miniconda/base/envs/py310/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


### Choose LLM

In [151]:
llm = SambaNovaEndpoint(
    base_url=os.getenv('BASE_URL'),
    project_id=os.getenv('PROJECT_ID'),
    endpoint_id=os.getenv('ENDPOINT_ID'),
    api_key=os.getenv('API_KEY'),
    model_kwargs={
        "do_sample": False, 
        "temperature": 0.0,
        "max_tokens_to_generate": 250
    },
)

### Retrieve information from vector db

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="question",
    output_key="response",
    return_source_documents=True,
)

custom_prompt_template = """
<s>[INST] <<SYS>>\nYou're an expert in filing reports\n<</SYS>>\n 
Given the following context enclosed in backticks regarding a company annual/quarterly report filing:
```
{context}
```
Consider the question:  
{question}
Answer the question using only the information from the context. If the answer to the question can't be extracted from the preovious context, then say "I do not have information regarding this".
Helpful Answer: [/INST]"""
CUSTOMPROMPT = PromptTemplate(
    template=custom_prompt_template, input_variables=["context", "question"]
)
## Inject custom prompt
qa.combine_documents_chain.llm_chain.prompt = CUSTOMPROMPT
query = "What's the name of the company of this filing report?"
qa(query)

### Q&A chatbot

The following image shows the general idea of the architecture for this retriever chatbot. </br>
More resources about how chatbots are implemented with langchain [here](https://python.langchain.com/docs/use_cases/chatbots)

![retriever_chatbot_architecture](https://python.langchain.com/assets/images/chat_use_case-eb8a4883931d726e9f23628a0d22e315.png )

### Chatbot example

In [19]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("<s>[INST] {question} [/INST]"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

conversation({"question": "hi, my name is Rodrigo"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history.
Human: <s>[INST] hi, my name is Rodrigo [/INST]

> Finished chain.


{'question': 'hi, my name is Rodrigo',
 'chat_history': [HumanMessage(content='hi, my name is Rodrigo'),
  AIMessage(content=" Assistant: Hello Rodrigo! Nice to meet you. Is there something you need help with or something you'd like to chat about?")],
 'text': " Assistant: Hello Rodrigo! Nice to meet you. Is there something you need help with or something you'd like to chat about?"}

In [20]:
conversation({"question": "can you tell me about the benefits of using AI in society?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history.
Human: hi, my name is Rodrigo
AI:  Assistant: Hello Rodrigo! Nice to meet you. Is there something you need help with or something you'd like to chat about?
Human: <s>[INST] can you tell me about the benefits of using AI in society? [/INST]

> Finished chain.


{'question': 'can you tell me about the benefits of using AI in society?',
 'chat_history': [HumanMessage(content='hi, my name is Rodrigo'),
  AIMessage(content=" Assistant: Hello Rodrigo! Nice to meet you. Is there something you need help with or something you'd like to chat about?"),
  HumanMessage(content='can you tell me about the benefits of using AI in society?'),
  AIMessage(content=' Assistant: Of course, Rodrigo! There are many benefits to using AI in society. Some of the most significant advantages include:\n\n1. Increased efficiency: AI can automate many repetitive and mundane tasks, freeing up time for people to focus on more important and creative work.\n2. Improved decision-making: AI can analyze vast amounts of data and provide insights that humans might miss, allowing for more informed')],
 'text': ' Assistant: Of course, Rodrigo! There are many benefits to using AI in society. Some of the most significant advantages include:\n\n1. Increased efficiency: AI can automate 

In [21]:
conversation({"question": "could you tell me more details about point 1?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history.
Human: hi, my name is Rodrigo
AI:  Assistant: Hello Rodrigo! Nice to meet you. Is there something you need help with or something you'd like to chat about?
Human: can you tell me about the benefits of using AI in society?
AI:  Assistant: Of course, Rodrigo! There are many benefits to using AI in society. Some of the most significant advantages include:

1. Increased efficiency: AI can automate many repetitive and mundane tasks, freeing up time for people to focus on more important and creative work.
2. Improved decision-making: AI can analyze vast amounts of data and provide insights that humans might miss, allowing for more informed
Human: <s>[INST] could you tell me more details about point 1? [/INST]

> Finished chain.


{'question': 'could you tell me more details about point 1?',
 'chat_history': [HumanMessage(content='hi, my name is Rodrigo'),
  AIMessage(content=" Assistant: Hello Rodrigo! Nice to meet you. Is there something you need help with or something you'd like to chat about?"),
  HumanMessage(content='can you tell me about the benefits of using AI in society?'),
  AIMessage(content=' Assistant: Of course, Rodrigo! There are many benefits to using AI in society. Some of the most significant advantages include:\n\n1. Increased efficiency: AI can automate many repetitive and mundane tasks, freeing up time for people to focus on more important and creative work.\n2. Improved decision-making: AI can analyze vast amounts of data and provide insights that humans might miss, allowing for more informed'),
  HumanMessage(content='could you tell me more details about point 1?'),
  AIMessage(content=' AI:  Assistant: Sure, Rodrigo! Point 1 refers to the ability of AI to automate repetitive and mundane 

In [22]:
conversation({"question": "Great, what was my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history.
Human: hi, my name is Rodrigo
AI:  Assistant: Hello Rodrigo! Nice to meet you. Is there something you need help with or something you'd like to chat about?
Human: can you tell me about the benefits of using AI in society?
AI:  Assistant: Of course, Rodrigo! There are many benefits to using AI in society. Some of the most significant advantages include:

1. Increased efficiency: AI can automate many repetitive and mundane tasks, freeing up time for people to focus on more important and creative work.
2. Improved decision-making: AI can analyze vast amounts of data and provide insights that humans might miss, allowing for more informed
Human: could you tell me more details about point 1?
AI:  AI:  Assistant: Sure, Rodrigo! Point 1 refers to the ability of AI to automate repetitive and mundane tasks. By using AI

{'question': 'Great, what was my name?',
 'chat_history': [HumanMessage(content='hi, my name is Rodrigo'),
  AIMessage(content=" Assistant: Hello Rodrigo! Nice to meet you. Is there something you need help with or something you'd like to chat about?"),
  HumanMessage(content='can you tell me about the benefits of using AI in society?'),
  AIMessage(content=' Assistant: Of course, Rodrigo! There are many benefits to using AI in society. Some of the most significant advantages include:\n\n1. Increased efficiency: AI can automate many repetitive and mundane tasks, freeing up time for people to focus on more important and creative work.\n2. Improved decision-making: AI can analyze vast amounts of data and provide insights that humans might miss, allowing for more informed'),
  HumanMessage(content='could you tell me more details about point 1?'),
  AIMessage(content=' AI:  Assistant: Sure, Rodrigo! Point 1 refers to the ability of AI to automate repetitive and mundane tasks. By using AI fo

### Chatbot with Summary and SEC retriever

In [112]:
def print_response(response: dict) -> None:
    
    print('\nQUESTION:')
    print(response['question'])
    
    print('\nCONTEXT:')
    for document in response['source_documents']:
        pprint(document.page_content, width=180)
        print()
    
    print('\nCHAT HISTORY:')
    for message in response['chat_history']:
        message.pretty_print()
        
    print('\nANSWER:')
    print(response['answer'])

In [180]:
custom_condensed_question_template = """<s>[INST] <<SYS>>\nYou're an assitant\n<</SYS>>
Given the following chat history and follow up question, rephrase the follow up question using the chat history. Only output the rephrased question.

Chat history:
{chat_history}

Follow up question: {question}

Rephrased question: [/INST]"""

custom_condensed_question_prompt = PromptTemplate.from_template(custom_condensed_question_template)

custom_qa_template = """
<s>[INST] <<SYS>>\nYou're a helpful assistant in filing reports. Follow the following rules:
1. If you don't know the answer, respond kindly that you don't have information about it. 
2. Do not try to make up an answer.
3. Start the conversation expressing general assitance in anything the user would like to ask.\n<</SYS>>
Given the following context and follow up question, provide a helpful answer.

Context:
{context}

Follow up question: {question}

Helpful answer: [/INST]"""

custom_qa_prompt = PromptTemplate.from_template(custom_qa_template)

memory = ConversationSummaryMemory(
    llm=llm, 
    max_token_limit=50,
    buffer="The human and AI greet each other.",
    output_key='answer',
    memory_key='chat_history',
    return_messages=True,
)

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

qa = ConversationalRetrievalChain.from_llm(
    llm, 
    retriever=retriever, 
    memory=memory, 
    chain_type="stuff",
    return_source_documents=True, 
    verbose=True,
    condense_question_prompt = custom_condensed_question_prompt,
    combine_docs_chain_kwargs={'prompt': custom_qa_prompt}
)

response = qa({"question": "what are the biggest risk that the company is facing?"})
print_response(response)



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You're an assitant
<</SYS>>
Given the following chat history and follow up question, rephrase the follow up question using the chat history. Only output the rephrased question.

Chat history:

system: The human and AI greet each other.

Follow up question: what are the biggest risk that the company is facing?

Rephrased question: [/INST]

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

<s>[INST] <<SYS>>
You're a helpful assistant in filing reports. Follow the following rules:
1. If you don't know the answer, respond kindly that you don't have information about it. 
2. Do not try to make up an answer.
3. Start the conversation expressing general assitance in anything the user would like to ask.
<</SYS>>
Given the following context and follow up question, provide a helpful answer.

Context:
(e.g., vehicle functionality, Autopilot and F

In [181]:
response = qa({"question": "Tell me about the business of the company, what products and services does it offer?"})
print_response(response)



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You're an assitant
<</SYS>>
Given the following chat history and follow up question, rephrase the follow up question using the chat history. Only output the rephrased question.

Chat history:

system: 
The human and AI greet each other. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human asks what the biggest risk the company is facing, and the AI mentions ongoing investigations by government agencies and data privacy concerns.

Follow up question: Tell me about the business of the company, what products and services does it offer?

Rephrased question: [/INST]

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

<s>[INST] <<SYS>>
You're a helpful assistant in filing reports. Follow the following rules:
1. If you don't know the answer, respond kindly that you don't ha

In [184]:
response = qa({"question": "tell me more about the products that Tesla offers based on the report"})
print_response(response)



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You're an assitant
<</SYS>>
Given the following chat history and follow up question, rephrase the follow up question using the chat history. Only output the rephrased question.

Chat history:

system: 
The human and AI greet each other. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human asks what the biggest risk the company is facing, and the AI mentions ongoing investigations by government agencies and data privacy concerns. The AI explains that the company takes data privacy very seriously and has implemented various measures to ensure compliance with privacy, data protection, and security laws and regulations. The company offers a range of electric vehicles, software, and energy products that are designed to be innovative, sustainable, and efficient, and it collects and uses customer data to provide these products and services. The A